In [1]:
%reload_ext autoreload
%autoreload 2

In [17]:
from datetime import datetime, timedelta
import pandas as pd

current_date = pd.to_datetime(datetime.utcnow()).floor('h') # - timedelta(hours=1)
current_date = current_date.tz_localize('UTC')
print(f'{current_date=}')

current_date=Timestamp('2024-04-29 18:00:00+0000', tz='UTC')


In [21]:
from src.inference import load_batch_of_features_from_store

features = load_batch_of_features_from_store(current_date)

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/674964
Connected. Call `.close()` to terminate connection gracefully.
Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/674964
Connected. Call `.close()` to terminate connection gracefully.
Finished: Reading data from Hopsworks, using ArrowFlight (13.22s) 
ts_data=                     pickup_hour  rides  pickup_location_id
105869 2024-04-01 18:00:00+00:00      0                   1
115634 2024-04-01 19:00:00+00:00      0                   1
159548 2024-04-01 20:00:00+00:00      0                   1
146478 2024-04-01 21:00:00+00:00      1                   1
26580  2024-04-01 22:00:00+00:00      0                   1
...                          ...    ...                 ...
49946  2024-04-29 13:00:00+00:00      4                 2

In [19]:
features.shape

(265, 674)

In [22]:
from src.inference import (
    load_model_from_registry,
    get_model_predictions
)

model = load_model_from_registry()
predictions = get_model_predictions(model, features)

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/674964
Connected. Call `.close()` to terminate connection gracefully.
[LightGBM] [Warning] feature_fraction is set=0.9100352710917206, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9100352710917206
[LightGBM] [Warning] bagging_fraction is set=0.8684737142064078, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8684737142064078


In [23]:
predictions['pickup_hour'] = current_date
predictions

,pickup_location_id,predicted_demand,pickup_hour
0,1,28.0,2024-04-29 18:00:00+00:00
1,2,27.0,2024-04-29 18:00:00+00:00
2,3,30.0,2024-04-29 18:00:00+00:00
3,4,34.0,2024-04-29 18:00:00+00:00
4,5,26.0,2024-04-29 18:00:00+00:00
...,...,...,...
260,261,29.0,2024-04-29 18:00:00+00:00
261,262,40.0,2024-04-29 18:00:00+00:00
262,263,21.0,2024-04-29 18:00:00+00:00
263,264,41.0,2024-04-29 18:00:00+00:00


In [25]:
from src.feature_store_api import get_feature_store
import src.config as config

# connect to the feature group
feature_group = get_feature_store().get_or_create_feature_group(
    name=config.FEATURE_GROUP_MODEL_PREDICTIONS,
    version=1,
    description="Predictions generate by our production model",
    primary_key = ['pickup_location_id', 'pickup_hour'],
    event_time='pickup_hour',
)

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/674964
Connected. Call `.close()` to terminate connection gracefully.


In [26]:
feature_group.insert(predictions, write_options={"wait_for_job": False})

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/674964/fs/670787/fg/766285


Uploading Dataframe: 0.00% |          | Rows 0/265 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: model_predictions_feature_group_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/674964/jobs/named/model_predictions_feature_group_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x254c0f22e80>, None)